In [2]:
import numpy as np
import io
import os
import pathlib

In [3]:
def open_input(filename):
    #read file
    path=str(pathlib.Path().absolute())+"/dataset/"+filename
    input_file = io.open(path, mode='r')
    Lines = input_file.readlines() #lista di righe del file sorgente
    input_file.close()

    #parsing first line
    [num_file,num_target,num_server] = np.fromstring(Lines[0], dtype=int, sep=' ')

    #conversion of read lines from list of string to list of lists 
    Lines = [i.strip("[]\n").split(" ") for i in Lines]

    #blank dicts
    time = {}
    dep = {}
    target = {}

    
    #build time & dep dicts
    row=1

    while row <= num_file*2:
        time.update({Lines[row][0] : [Lines[row][1], Lines[row][2]]})
        row_dep=row+1

        if Lines[row_dep][0]== "0":
            dep.update({Lines[row][0]: []})
        else:
            num_dep=int(Lines[row_dep][0])
            dep.update({Lines[row][0]: Lines[row_dep][-num_dep:]})

        row+=2

        
    #build target dict     
    row=1+num_file*2

    while row <= num_target+(num_file*2):
        target.update({Lines[row][0] : [Lines[row][1], Lines[row][2]]})
        row+=1;

    #conversion dicts lists from string to int
    time = dict((k,list(map(int,v))) for k,v in time.items())
    target = dict((k,list(map(int,v))) for k,v in target.items())

    #generate file list
    file=list(time.keys())

    return num_file, num_target, num_server, file, time, dep, target

In [12]:
#creates the matrix of execution sequences
def create_sequence(num_server, target, dep): 
    
    #INPUT
    #num_serv: number of servers
    #target: dictionary of target's files
    #dep: dictionary of file's dependencies
    
    #OUTPUT
    #matrix: matrix of execution sequences
    
    #number of files target
    num_target = len(target)
    #list of target files
    target_keys = list(target.keys())
    #declaration of emtpy matrix
    matrix = []
    
   
    matrix_dim = min(num_server,num_target)
    #insert a target file for each matrix' row 
    for i in range(matrix_dim):
        row = []
        row.append(target_keys[i])
        matrix.append(row)
    
    #declaration of empty list of nodes. Contains nodes that must be inserted in a matrix' row
    nodes = []
    #declaration of empty list of files. Contains files that have already been inserted in matrix
    file_list = []
    
    #continues filling the matrix with dependencies of file targets
    for i in range(matrix_dim):
        nodes.append(matrix[i][0])
        
        while len(nodes) != 0:
            dep_list = dep[nodes[0]]
            
            for f in dep_list:
                #check whether the dependency file is already being compiled on a server or not
                if not(f in file_list):
                    matrix[i].insert(0,f)
                    file_list.append(f)
                    nodes.append(f)
                    
            nodes.pop(0)  
        
    return matrix

#creates list of target files that do not generates dependencies
def find_target(target, dep):
    no_dep_tar = []
    
    for key_tar in target:
        no_dep = True
        
        for key_dep in dep:
            if key_tar in dep[key_dep]:
                no_dep = False
                break
                
        if no_dep:
            no_dep_tar.append(key_tar)
    return no_dep_tar

In [13]:
num_file, num_target, num_server, file, time, dep, target = open_input("a_example.in")
matrix = create_sequence(num_server, target, dep)
matrix

[['c1', 'c3'], ['c0', 'c2', 'c4']]

# DOUBTS
* Ho fatto in modo da dare alla matrice tante righe per quanti sono il minimo di target o server. Se però ho più server che target alcuni server perdono tempo senza lavorare.
* Ho preso i file target da assegnare in ordine di come si presentavano all'interno della lista target. Non sono convinto del fatto che usando invece i file target senza dipendenze le prestazioni siano migliori. Bisogna inoltre tenere in conto che se scegliessimo questa strada avremmo ancora meno server a lavorare (inoltre va modificato il codice).
* La soluzione che trovo in questo caso, con due server, non è una soluzione ottima ovviamente ma non sono sicuro che sia ammissibile. In che senso la soluzione trovata dev'essere ammissibile?
* Non sono sicuro che il codice abbia una complessità decente.